# **Lab Session : Feature selection II**

Here, we will continue with work of the notebook Feature Selection I, and we will focus on wrapper and embedded feature selection methods. To analyze their performances we will continue working with the Olivetti face detection problem. 

#### ** During this lab we will cover: **
#### *Part 1: Wrapers for feature selection: RFE* 
#### *Part 2: Embedded methods: L1 regularization*

In [1]:
%matplotlib inline

## ** Part 0.1: Download and prepare the data **

Olivetti dataset consists of ten different images of each of 40 distinct subjects. For some subjects, the images were taken at different times, varying the lighting, facial expressions (open / closed eyes, smiling / not smiling) and facial details (glasses / no glasses). All the images were taken against a dark homogeneous background with the subjects in an upright, frontal position (with tolerance for some side movement).
    
The next code includes the lines to download this data set and create the training, validation and test data partitions, as well as normalize them.

In [23]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_olivetti_faces 
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler


###############################################################################
# Download the data, if not already on disk and load it as numpy arrays
print 'The first time that you downlaod the data it can take a while...'
olivetti_people = fetch_olivetti_faces()

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = olivetti_people.images.shape

# for machine learning we use the data directly (as relative pixel
# positions info is ignored by this model)
X = olivetti_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
Y = olivetti_people.target
n_classes = np.unique(Y).shape[0]

print("Dataset size information:")
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)


###############################################################################
# Preparing the data

# Initialize the random generator seed to compare results
np.random.seed(1)

# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

# split into a training and validation set
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.333)

# Normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Binarize the labels for some feature selection methods
set_classes = np.unique(Y)
Y_train_bin = label_binarize(Y_train, classes=set_classes)

print("Number of training samples: %d" % X_train.shape[0])
print("Number of validation samples: %d" % X_val.shape[0])
print("Number of test samples: %d" % X_test.shape[0])

The first time that you downlaod the data it can take a while...
Dataset size information:
n_features: 4096
n_classes: 40
Number of training samples: 200
Number of validation samples: 100
Number of test samples: 100


## ** Part 0.2: Some utilities **

Here, you can find some functions from the notebook Feature Selection I which can be usefull for completing the following sections.

In [5]:
from sklearn import svm

def SVM_accuracy_evolution(X_train_s, Y_train, X_val_s, Y_val, X_test_s, Y_test, rang_feat):
    """Compute the accuracy of training, validation and test data for different the number of features given
        in rang_feat.

    Args:
        X_train_s (numpy dnarray): training data sorted by relevance (more relevant are first) (number data x number dimensions).
        Y_train (numpy dnarray): labels of the training data (number data x 1).
        X_val_s (numpy dnarray): validation data sorted by relevance (more relevant are first) (number data x number dimensions).
        Y_val (numpy dnarray): labels of the validation data (number data x 1).
        X_test_s (numpy dnarray): test data sorted by relevance (more relevant are first) (number data x number dimensions).
        Y_test (numpy dnarray): labels of the test data (number data x 1).
        rang_feat: range with different number of features to be evaluated                                           
   
    """
    
    # Define the model to train a liner SVM and adjust by CV the parameter C
    clf = svm.SVC(kernel='linear')
    acc_tr = []
    acc_val = []
    acc_test = []
    for i in rang_feat:
        # Train SVM classifier
        clf.fit(X_train_s[:, :i], Y_train)
        # Compute accuracies
        acc_tr.append(clf.score(X_train_s[:, :i], Y_train))
        acc_val.append(clf.score(X_val_s[:, :i], Y_val))
        acc_test.append(clf.score(X_test_s[:, :i], Y_test))

    return np.array(acc_tr), np.array(acc_val), np.array(acc_test)


In [6]:
import matplotlib.pyplot as plt

def plot_accuracy_evolution(rang_feat, acc_tr, acc_val, acc_test):

    """Plot the accuracy evolution for training, validation and test data sets.
    Args:
        rang_feat: range with different number of features where the accuracy has been evaluated   
        acc_tr: numpy vector with the training accuracies
        acc_val: numpy vector with the validation accuracies
        acc_test: numpy vector with the test accuracies                                          
    """

    plt.plot(rang_feat, acc_tr, "b", label="train")
    plt.plot(rang_feat, acc_val, "g", label="validation")
    plt.plot(rang_feat, acc_test, "r", label="test")
    plt.xlabel("Number of features")
    plt.ylabel("Accuracy")
    plt.title('Accuracy evolution')
    plt.legend(['Training', 'Validation', 'Test'], loc = 4)


In [9]:
import matplotlib.pyplot as plt
def plot_image(image, h, w):
    """Helper function to plot a face image
    Args:
        image: numpy vector with the image to plot (of dimensions h*w)
        h: height of the image (in number of pixels)
        w: width of the image (in number of pixels)  """  
    
    plt.imshow(image.reshape((h, w)), cmap=plt.cm.gray)
    plt.xticks(())
    plt.yticks(())
    plt.colorbar()

### *Part 1: Wrapers for feature selection: RFE* 

The Recursive Feature Elimination (RFE) method iteratively trains a set of SVM classifier and, in each step, it eliminates a feature (or a subset of features) in such a way that the classification margin is reduced the least.

Scikit-Learn provides a function with a full implementation of the RFE method. This function, [RFE( )](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html), let user select the classifier to consider, the number of selected features (parameter n_features_to_select) and the number of features removed in each step (parameter step).  As result, in parameter .ranking returns the ranking position of the each feature (i.e., .ranking\_[i] corresponds to the ranking position of the i-th feature).

Complete the following code to:
* Train the RFE approach with a linear SVM fixing  n_features_to_select to 1 (so that we run the RFE method until the end obtaining a full raking with all the features) and step to 10 (just to speed up the training).
* Use the content of .ranking\_ to obtain a ranking with the most relevant features, save their positions in the variable  ind\_rel\_feat.


In [ ]:
###########################################################
# TODO: Replace <FILL IN> with appropriate code
###########################################################
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

print 'The training of this method can take some minutes, be patient...'
# Define the classifier, the RFE method and train it
estimator = SVC(kernel="linear")
RFE_selector = RFE(#<FILL IN>)
RFE_selector.fit(X_train, Y_train)

# Obtain the positions of the sorted features (the most relevant first)
ind_rel_feat = # <FILL IN>

# Print the feature ranking
print("Feature ranking:")

for f in range(10):
    print("%d. feature %d" % (f + 1, ind_rel_feat[f]))

In [ ]:
###########################################################
# TEST CELL
###########################################################

from test_helper import Test

# TEST Training and test data generation
Test.assertEquals(np.sum(ind_rel_feat[:5]- [781, 3615, 648,  1016, 51]),0, 'incorrect result: first positions of relevant features are uncorrect')

As in the random forest section, use the above the result to:
* Evaluate the linear SVM performance with the most relevant features are used (use SVM_accuracy_evolution( ) function). Plot the evolution of the training, validation and test accuracies.
* Obtain, using the validation accuracy, the optimum number of features to use.
* Create the mask with the selected features and plot it (you can use the plot_image( ) function).

In [ ]:
###########################################################
# TODO: Replace <FILL IN> with appropriate code
###########################################################
# Define the range of features to explore
# (explore the 1000 most relevant ones, starting with 5 and usings steps of 10 features)
rang_feat = np.arange(5, 1000, 10) # To speed up the execution, we use steps of 10 features.
[acc_tr, acc_val, acc_test] = SVM_accuracy_evolution(#<FILL IN>)

# Plot it!
plt.figure()
plot_accuracy_evolution(rang_feat, acc_tr, acc_val, acc_test)
plt.show()

# Find the optimum number of features
num_opt_feat = # <FILL IN>
test_acc_opt = # <FILL IN>

print 'Number optimum of features: ' + str(num_opt_feat)
print("The optimum test accuracy is  %2.2f%%" %(100*test_acc_opt))

# Create and plot the mask
mask = # <FILL IN>
plt.figure()
plot_image(mask, h, w)
plt.show()

In [ ]:
###########################################################
# TEST CELL
###########################################################

from test_helper import Test

# TEST Training and test data generation
Test.assertEquals(num_opt_feat,395, 'incorrect result: number of selected features is uncorrect')
Test.assertEquals(test_acc_opt,0.88, 'incorrect result: test error is uncorrect')

## ** Part 2: Embedded methods:  L1 regularization**

Embedded methods are able to carry out the feature selection process during the classifier training, so both stages (feature selection and classifier training) are completely linked and, therefore, the selection process is guided by the classifier.

In this case, we are going to study two well-known techniques:  
* L1 SVM: in this case we can train linear SVM regularized with a L1 penalty which is able to provide sparsity over the weights vector. As to obtain the classifier output we have to multiply the input data by this vector, those input features associated to the zeros of the weight vector are not used during the classification process (at least, in binary problem, we later analyze this in detail for the multiclass case). 

* L1 Logistic Regression: here, we will apply the above L1 penalty, but using a Logistic Regression classifier.

### ** Part 2.1: L1-SVM**

In this last section, we are going to analyze the properties of the L1-SVM as feature selection approach. For this purpose, we are going to use the linear SVM implementation given by the method LinearSVC, which directly let us select the regularization to be used.

Unlike previous methods, here we cannot obtain a ranking of variables. In this case, we have to sweep the value of the regularization parameter in order to get a higher (or lower) sparsity of the weight vector. According to this, complete the following code to:
* train a linear l1-SVM for different values of the regularization parameter, for each value obtain the training, validation and test accuracies, as well as the sparsity rate. 
* Plot the resulting accuracy curves.
* Finally, analyzing the validation accuracy, select the optimum value of C.

Note: Sparsity Rate (SR) is defined as the mean number of zeros of the SVM weight vector (check the parameter .coef\_ of the SVM classifier), then, if all elements are zero, SR has to be 100%. In our case, that we are working with a multiclass problem, we have a set of vectors, so we have to average the number of zeros overall the vectors.

In [ ]:
###########################################################
# TODO: Replace <FILL IN> with appropriate code
###########################################################
from sklearn.svm import LinearSVC
np.random.seed(1)
# Defining some useful variables to save results
acc_tr = []
acc_val = []
acc_test = []
sparsity_rate = []

# Defining the range of C values to explore
rang_C = 2*np.logspace(-2, 2, 20)

print 'The training of this method can take some minutes, be patient...'
for i, C in enumerate(rang_C):
    # Define and train SVM classifier
    svm_l1 = LinearSVC(C=C, penalty="l1", dual=False)
    svm_l1.fit(X_train, Y_train)
    
    # Compute the sparsity rate (coef_l1 contains zeros due to the
    # L1 sparsity inducing norm)
    
    sparsity_rate.append(# <FILL IN>)
    
    # Compute accuracies
    acc_tr.append(#<FILL IN>)
    acc_val.append(#<FILL IN>)
    acc_test.append(#<FILL IN>)

    
# Plot the accuracy curves
plt.figure()
plot_accuracy_evolution(sparsity_rate, acc_tr, acc_val, acc_test)
plt.xlabel("Sparsity rate")
plt.show()

# Find the optimum value of C
C_opt = # <FILL IN>
test_acc_opt = # <FILL IN>

print 'Optimum value of C: ' + str(C_opt)
print("The optimum test accuracy is  %2.2f%%" %(100*test_acc_opt))


In [ ]:
###########################################################
# TEST CELL
###########################################################

from test_helper import Test

# TEST Training and test data generation
Test.assertEquals(np.round(C_opt,2), 2.07, 'incorrect result: the selected value of C is uncorrect')
Test.assertEquals(test_acc_opt,0.9, 'incorrect result: test error is uncorrect')

Until now, the L1-SVM has provided a sparsity vector instead of a real feature selection. Note that, as we are working in a multiclass problem, we have as many vectors as categories (we are training a 1 vs. rest classifier) and each vector has the zeros in different positions, so each classifier is using different input features. So, we obtain a feature selection per categories.

To analyze this selection per categories, you can run the following code which will train again the L1-SVM with optimum value of the C that we have just selected. Then, it will let you get the set of weight vectors and generate a set of masks with the selected variables for each class. Finally, plot all these masks.

In [ ]:
# Train the linear SVM with the optimum value of C
svm_l1 = LinearSVC(C=C_opt, penalty="l1", dual=False)
svm_l1.fit(X_train, Y_train)
coef_l1 = svm_l1.coef_
coef_l1.shape

#### Useful function for display purposes ##########
import matplotlib.pyplot as plt
def plot_gallery(images, titles, h, w, n_row=4, n_col=10):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(images.shape[0]):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())
##################################################### 

# Create a mask per class
mask_class = np.zeros(coef_l1.shape)
mask_class [np.where(coef_l1>0)] = 1

# Plot all the masks
titles = ['class '+str(c) for c in set_classes]
plot_gallery(mask_class, titles, h, w)
plt.show()


Finally, we can convert this feature selection per class in a real feature selection. For this purpose, we just have to select all the input features which are used in at least one individual classifier. 
Compete the following code to be able to:
* Find the selected features (those used by at least one individual classifier). 
* Save in num\_var\_sel the number of selected features.
* Train a linear SVM with these features and compute its test accuracy.
* Plot the mask of selected features

If we define the feature relevance as the number of classifiers that use that feature, could you obtain a mask of feature relevances?  

In [ ]:
###########################################################
# TODO: Replace <FILL IN> with appropriate code
###########################################################
from sklearn import svm

# Finding the selected fatures
ind_var_sel = # <FILL IN>

# Save the number of selected features
num_var_sel = # <FILL IN>

# Evaluating performance (for comparison purposes, let's use SVC classifier)
clf = svm.SVC(kernel='linear')
clf.fit(# <FILL IN>)
acc_test = # <FILL IN>

print 'Number optimum of features: ' + str(num_var_sel )
print("The test accuracy with real feature selection is  %2.2f%%" %(100*acc_test))

# Obtain and plot the mask
mask = # <FILL IN>
plot_image(mask, h, w)
plt.show()
    
#Plot a relevance mask
# <FILL IN>

### ** Part 2.2: L1-Logistic Regression**

The Scikit-Learn Logistic Regression implementation lets us add an L1 regularization to obtain sparsity over the weigths vector, see help

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

So, using the L1-SVM code as re